In [1]:
%%javascript
/**********************************************************************************************
Known Mathjax Issue with Chrome - a rounding issue adds a border to the right of mathjax markup
https://github.com/mathjax/MathJax/issues/1300
A quick hack to fix this based on stackoverflow discussions: 
http://stackoverflow.com/questions/34277967/chrome-rendering-mathjax-equations-with-a-trailing-vertical-line
**********************************************************************************************/

$('.math>span').css("border-left-color","transparent")

<IPython.core.display.Javascript object>

In [ ]:
%reload_ext autoreload
%autoreload 2

# DATASCI W261 - Machine Learning At Scale
## Assignment - Week 09
---

* **Name:**  Megan Jasek
* **Email:**  meganjasek@ischool.berkeley.edu
* **Class Name:**  W261-2
* **Date:**  7/17/16

---
### Instructions

Due by 07/17/2016

[Submission Link - Google Form](https://docs.google.com/forms/d/1ZOr9RnIe_A06AcZDB6K1mJN4vrLeSmS2PD6Xm3eOiis/viewform?usp=send_form) 

### Documents:
* IPython Notebook, published and viewable online.
* PDF export of IPython Notebook.
    
### Useful References

* Data-intensive text processing with MapReduce. San Rafael, CA: Morgan & Claypool Publishers. Chapter 5. 


---

<h2 style="color:darkblue">HW 9 Dataset</h2>

Note that all referenced files life in the enclosing directory. [Checkout the Data subdirectory on Dropbox](https://www.dropbox.com/sh/2c0k5adwz36lkcw/AAAAKsjQfF9uHfv-X9mCqr9wa?dl=0) or the AWS S3 buckets (details contained each question). 

<h2 style="color:darkblue"> HW 9.0: Short answer questions </h2>

__ What is PageRank and what is it used for in the context of web search?__



<hr>

__ What modifications have to be made to the webgraph in order to leverage the machinery of Markov Chains to compute the Steady State Distibution? __



<hr>

__ OPTIONAL: In topic-specific pagerank, how can we ensure that the irreducible property is satifsied? (HINT: see HW9.4) __



<hr>


<h2 style="color:darkblue"> HW 9.1: MRJob implementation of basic PageRank </h2>

Write a basic MRJob implementation of the iterative PageRank algorithm that takes sparse adjacency lists as input (as explored in HW 7).

Make sure that you implementation utilizes teleportation (1-damping/the number of nodes in the network), and further, distributes the mass of dangling nodes with each iteration so that the output of each iteration is correctly normalized (sums to 1).


[NOTE: The PageRank algorithm assumes that a random surfer (walker), starting from a random web page, chooses the next page to which it will move by clicking at random, with probability d,one of the hyperlinks in the current page. This probability is represented by a so-called *damping factor* d, where d ∈ (0, 1). Otherwise, with probability (1 − d), the surfer jumps to any web page in the network. If a page is a dangling end, meaning it has no outgoing hyperlinks, the random surfer selects an arbitrary web page from a uniform distribution and “teleports” to that page]


As you build your code, use the test data:

> s3://ucb-mids-mls-networks/PageRank-test.txt

Or under the Data Subfolder for HW7 on Dropbox with the same file name. 
> Dropbox: https://www.dropbox.com/sh/2c0k5adwz36lkcw/AAAAKsjQfF9uHfv-X9mCqr9wa?dl=0

with teleportation parameter set to 0.15 (1-d, where d, the damping factor is set to 0.85), and crosscheck your work with the true result, displayed in the first image in the [Wikipedia article](https://en.wikipedia.org/wiki/PageRank)
and here for reference are the corresponding PageRank probabilities:
<pre>

A, 0.033
B, 0.384
C, 0.343
D, 0.039
E, 0.081
F, 0.039
G, 0.016
H, 0.016
I, 0.016
J, 0.016
K, 0.016

</pre>

<h2 style="color:darkgreen"> HW 9.1 Implementation </h2>

### Create test file:  PageRank-test.txt

In [3]:
%%writefile PageRank-test.txt
B	{'C': 1}
C	{'B': 1}
D	{'A': 1, 'B': 1}
E	{'D': 1, 'B': 1, 'F': 1}
F	{'B': 1, 'E': 1}
G	{'B': 1, 'E': 1}
H	{'B': 1, 'E': 1}
I	{'B': 1, 'E': 1}
J	{'E': 1}
K	{'E': 1}

Writing PageRank-test.txt


### Expected Output from PageRank-test.txt

https://en.wikipedia.org/wiki/PageRank

and here for reference are the corresponding PageRank probabilities:

A, 0.033  
B, 0.384  
C, 0.343  
D, 0.039  
E, 0.081  
F, 0.039  
G, 0.016  
H, 0.016  
I, 0.016  
J, 0.016  
K, 0.016  

0.999 = 0.033+0.384+0.343+0.039+0.081+0.039+0.016+0.016+0.016+0.016+0.016

### Create test file:  Lecture9_10_test.txt

In [2]:
%%writefile Lecture9_10_test.txt
N1	{'N2': 1, 'N4': 1}
N2	{'N3': 1, 'N5': 1}
N3	{'N4': 1}
N4	{'N5': 1}
N5	{'N1': 1, 'N2': 1, 'N3': 1}

Writing Lecture9_10_test.txt


### Expected Output from Lecture9_10_test.txt

Iteration 10  
N1 [{u'N2': 1, u'N4': 1}, 0.10593707133058988]  
N2 [{u'N3': 1, u'N5': 1}, 0.1575895919067216]  
N3 [{u'N4': 1}, 0.18432891803840884]  
N4 [{u'N5': 1}, 0.23511445473251036]  
N5 [{u'N1': 1, u'N2': 1, u'N3': 1}, 0.31702996399176964]  
Total Error:  0.00860982510288  

### Create test file:  Lecture9_10d_test.txt

In [46]:
%%writefile Lecture9_10d_test.txt
N1	{'N2': 1, 'N4': 1}
N2	{'N3': 1, 'N5': 1}
N3	{'N4': 1}
N4	{'N5': 1, 'N6': 1}
N5	{'N1': 1, 'N2': 1, 'N3': 1}

Writing Lecture9_10d_test.txt


In [70]:
%%writefile PageRankInit.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import ast
import json

# This class takes an adjacency list as input and outputs and initialized work file with 
# the following format:
# node_number \t [adjacency_list, infinity, '', 'U']
# One starting node is defined as an argument passed in to the class.  The default is node '1'.
# That starting node has the format:
# node_number \t [adjacency_list, 0.0, '', 'Q']
# U means Unvisited
# Q means Queued in the queue
class MRPageRankInit(MRJob):
    def configure_options(self):
        # Configure a new command line option to capture the stop_index for the shortest path
        super(MRPageRankInit, self).configure_options()
        self.add_passthrough_option('--platform', type='str', default='hadoop')

    def __init__(self, *args, **kwargs):
        super(MRPageRankInit, self).__init__(*args, **kwargs)
        self.total_nodes_calculated = False
        self.total_nodes = 0

    # For the node equal to the start index, yield
    #    node_number \t tuple(adj_list, 0.0, "", 'Q')
    # For all other nodes, yield
    #    node_number \t tuple(adj_list, infinity, "", 'U')    
    def mapper(self, _, line):
        node_num, adj_dict = line.strip().split('\t')
        adj_dict = ast.literal_eval(adj_dict)
        self.total_nodes += 1
        yield node_num, tuple((adj_dict, 1.0))
        yield '*' + node_num, 1
        for node in adj_dict:
            yield node, tuple(({}, 1.0))
            yield '*' + node, 1

    def reducer(self, node_num, data):
        if node_num[0] == '*':
            self.total_nodes += 1
        else:
            if self.total_nodes_calculated == False:
                yield '*total_nodes', self.total_nodes
                self.total_nodes_calculated = True
            f_adj_dict = {}
            for adj_dict, value in data:
                if adj_dict != {} and f_adj_dict == {}:
                    f_adj_dict = adj_dict
            print node_num, tuple((f_adj_dict, 1.0/self.total_nodes))
            yield node_num, tuple((f_adj_dict, 1.0/self.total_nodes))
        
    # Create the steps for the MRJob.  There is only a mapper in this job.
    # Run with 1 reducer when not running in Hadoop
    def steps(self):
        JOBCONF_STEP_LOCAL = {
            'mapreduce.job.reduces': '1'
        }
        if self.options.platform == 'local':
            JOBCONF_STEP = JOBCONF_STEP_LOCAL
        else:
            JOBCONF_STEP = {}
        return [
            MRStep(jobconf=JOBCONF_STEP,
                   mapper=self.mapper,
                   reducer=self.reducer)
               ]
            
if __name__ == '__main__':
    MRPageRankInit.run()

Overwriting PageRankInit.py


In [71]:
#!python PageRankInit.py Lecture9_10_test.txt '--platform=local'
#!python PageRankInit.py PageRank-test.txt '--platform=local'
!python PageRankInit.py Lecture9_10d_test.txt '--platform=local'

No configs found; falling back on auto-configuration
Creating temp directory /tmp/PageRankInit.hadoop.20160714.065856.164552
Running step 1 of 1...
N1 ({u'N2': 1, u'N4': 1}, 0.16666666666666666)
N2 ({u'N3': 1, u'N5': 1}, 0.16666666666666666)
N3 ({u'N4': 1}, 0.16666666666666666)
N4 ({u'N5': 1, u'N6': 1}, 0.16666666666666666)
N5 ({u'N1': 1, u'N2': 1, u'N3': 1}, 0.16666666666666666)
N6 ({}, 0.16666666666666666)
Streaming final output from /tmp/PageRankInit.hadoop.20160714.065856.164552/output...
"*total_nodes"	6
"N1"	[{"N2": 1, "N4": 1}, 0.16666666666666666]
"N2"	[{"N3": 1, "N5": 1}, 0.16666666666666666]
"N3"	[{"N4": 1}, 0.16666666666666666]
"N4"	[{"N5": 1, "N6": 1}, 0.16666666666666666]
"N5"	[{"N1": 1, "N2": 1, "N3": 1}, 0.16666666666666666]
"N6"	[{}, 0.16666666666666666]
Removing temp directory /tmp/PageRankInit.hadoop.20160714.065856.164552...


In [17]:
%%writefile PageRankStep1.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import json

class MRPageRankStep1(MRJob):
    def configure_options(self):
        # Configure a new command line option to capture the stop_index for the shortest path
        super(MRPageRankStep1, self).configure_options()
        self.add_passthrough_option('--num_converter', type='int', default=1)

    def __init__(self, *args, **kwargs):
        super(MRPageRankStep1, self).__init__(*args, **kwargs)
        self.total_page_rank_dangling = 0.0
    
    def mapper(self, key, line):
        node_num, data = line.strip().split('\t')
        node_num = node_num.strip('"')
        #print node_num, data
        if node_num != '*total_nodes':
            data = json.loads(data)
            adj_dict = data[0]
            degree = float(len(adj_dict))
            page_rank = data[1]
            # If the node is a dangling node, then add its page rank to the dangling mass count
            if adj_dict == {}:
                self.total_page_rank_dangling += page_rank
            # Yield the node to preserve the graph structure
            yield node_num, tuple((adj_dict, 0.0))
            for node, value in adj_dict.iteritems():
                yield node, tuple(({}, page_rank/degree))
    
    def mapper_final(self):
        print self.options.num_converter
        yield '*total_dangling', self.total_page_rank_dangling
    
    def reducer_test(self, node, data):
        print node
        yield node, None
    
    def reducer(self, node, data):
        print node
        if node == '*total_dangling':
            for d in data:
                self.total_page_rank_dangling += float(d)
            print 'dm: ', self.total_page_rank_dangling
            self.increment_counter('PageRank_Counters', 'Dangling_Mass', 
                                   int(self.options.num_converter*self.total_page_rank_dangling))
        else:
            f_adj_dict = {}
            f_page_rank = 0.0
            for adj_dict, page_rank in data:
                if adj_dict != {} and f_adj_dict == {}:
                    f_adj_dict = adj_dict
                f_page_rank += float(page_rank)
            yield node, tuple((f_adj_dict, f_page_rank))

    def steps(self):
        return [
            MRStep(mapper=self.mapper, mapper_final=self.mapper_final)
               ]
            
if __name__ == '__main__':
    MRPageRankStep1.run()

Overwriting PageRankStep1.py


In [16]:
iterations = 1
total_nodes = 11.0
#total_nodes = 15192277.0

# Set the damping factor
damping_factor = 0.85
num_converter = 10000000000
!hdfs dfs -rm -r /user/hadoop/HW9Output/PageRank1
#!hdfs dfs -rm -r /user/hadoop/HW9Input/PageRank1
#!hdfs dfs -cp /user/hadoop/HW9Results/PageRankInit_test /user/hadoop/HW9Input/PageRank1
#!python PageRankInit.py Lecture9_10d_test.txt '--platform=local'
!python PageRankStep1.py hdfs:///user/hadoop/HW9Input/PageRank1 -r hadoop '--num_converter=10000000000'

16/07/14 20:28:35 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `/user/hadoop/HW9Output/PageRank1': No such file or directory
No configs found; falling back on auto-configuration
Looking for hadoop binary in /usr/local/hadoop/bin...
Found hadoop binary: /usr/local/hadoop/bin/hadoop
Creating temp directory /tmp/PageRankStep1.hadoop.20160715.012836.244060
Using Hadoop version 2.7.1
Copying local files to hdfs:///user/hadoop/tmp/mrjob/PageRankStep1.hadoop.20160715.012836.244060/files/...
Looking for Hadoop streaming jar in /usr/local/hadoop...
Found Hadoop streaming jar: /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar
Running step 1 of 1...
  Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
  packageJobJar: [/tmp/hadoop-unjar3854810740116646126/] [] /tmp/streamjob369127782027282518.jar tmpDir=null
  Connecting to ResourceManager a

In [10]:
!hdfs dfs -cat hdfs:///user/hadoop/HW9Input/PageRank1/part-00000

16/07/14 20:03:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
"*total_nodes"	11
"A"	[{}, 0.09090909090909091]
"B"	[{"C": 1}, 0.09090909090909091]
"C"	[{"B": 1}, 0.09090909090909091]
"D"	[{"A": 1, "B": 1}, 0.09090909090909091]
"E"	[{"B": 1, "D": 1, "F": 1}, 0.09090909090909091]
"F"	[{"B": 1, "E": 1}, 0.09090909090909091]
"G"	[{"B": 1, "E": 1}, 0.09090909090909091]
"H"	[{"B": 1, "E": 1}, 0.09090909090909091]
"I"	[{"B": 1, "E": 1}, 0.09090909090909091]
"J"	[{"E": 1}, 0.09090909090909091]
"K"	[{"E": 1}, 0.09090909090909091]


In [4]:
%%writefile PageRankStep2.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import json

class MRPageRankStep2(MRJob):
    def configure_options(self):
        # Configure a new command line option to capture the stop_index for the shortest path
        super(MRPageRankStep2, self).configure_options()
        self.add_passthrough_option('--damping_factor', type='float', default=0.15)
        self.add_passthrough_option('--total_nodes', type='float', default=1.0)
        self.add_passthrough_option('--dangling_mass', type='float', default=0.0)

    def mapper(self, key, line):
        node_num, data = line.strip().split('\t')
        node_num = node_num.strip('"')
        print node_num, data
        if node_num != '*total_nodes':
            data = json.loads(data)
            adj_dict = data[0]
            page_rank = data[1]
            f_page_rank = (1-self.options.damping_factor)*(1.0/self.options.total_nodes)+ \
                self.options.damping_factor*((self.options.dangling_mass/self.options.total_nodes)+page_rank)
            print node_num, tuple((adj_dict, f_page_rank))                    
            yield node_num, tuple((adj_dict, f_page_rank))

    def steps(self):
        return [
            MRStep(mapper=self.mapper)
               ]
            
if __name__ == '__main__':
    MRPageRankStep2.run()

Overwriting PageRankStep2.py


In [4]:
import PageRankStep1
import PageRankStep2
import PageRankInit
reload(PageRankStep1)
reload(PageRankStep2)
reload(PageRankInit)
import json

# Set the damping factor
damping_factor = 0.85
#damping_factor = 1.0
epsilon = 0.01
iterations = 30
num_converter = 10000000000

# Set the name of the file that gets passed from iteration to iteration
work_filename = 'work_table.txt'

# Initialize a list ot store the page_ranks from iteration to iteration
page_ranks = []
# Call the PageRankInit MRJob
#mr_job = PageRankInit.MRPageRankInit(args=['Lecture9_10_test.txt', '--platform', 'local'])
#mr_job = PageRankInit.MRPageRankInit(args=['Lecture9_10d_test.txt', '--platform', 'local'])
mr_job = PageRankInit.MRPageRankInit(args=['PageRank-test.txt', '--platform', 'local'])
with mr_job.make_runner() as runner, open(work_filename, 'w') as f: 
    runner.run()
    # stream_output: get access of the output 
    for line in runner.stream_output():
        key, value =  mr_job.parse_output_line(line)
        # Store the total_nodes
        if key == '*total_nodes':
            total_nodes = float(value)
        else:
            page_ranks.append(value[1])
        f.write(key+'\t'+json.dumps(value)+'\n')

work_filename_1 = 'work_table_1.txt'
work_filename_2 = 'work_table_2.txt'
i = 0
while(i < iterations):
    # Run the PageRankStep1 MRJob
    mr_job = PageRankStep1.MRPageRankStep1(args=[work_filename, '--num_converter', str(num_converter)])
    with mr_job.make_runner() as runner, open(work_filename_1, 'w') as f: 
        runner.run()
        # stream_output: get access of the output 
        for line in runner.stream_output():
            key, value =  mr_job.parse_output_line(line)
            f.write(key+'\t'+json.dumps(value)+'\n')
    dangling_mass = runner.counters()[0]['PageRank_Counters']['Dangling_Mass']/float(num_converter)
    print 'dm', dangling_mass

    # Run the PageRankStep2 MRJob
    mr_job = PageRankStep2.MRPageRankStep2(args=[work_filename_1, '--damping_factor', damping_factor, '--total_nodes',
                                                total_nodes, '--dangling_mass', dangling_mass])
    with mr_job.make_runner() as runner, open(work_filename_2, 'w') as f: 
        runner.run()
        # stream_output: get access of the output 
        for line in runner.stream_output():
            key, value =  mr_job.parse_output_line(line)
            f.write(key+'\t'+json.dumps(value)+'\n')

    !mv $work_filename_2 $work_filename
    i += 1

A ({}, 0.09090909090909091)
B ({u'C': 1}, 0.09090909090909091)
C ({u'B': 1}, 0.09090909090909091)
D ({u'A': 1, u'B': 1}, 0.09090909090909091)
E ({u'B': 1, u'D': 1, u'F': 1}, 0.09090909090909091)
F ({u'B': 1, u'E': 1}, 0.09090909090909091)
G ({u'B': 1, u'E': 1}, 0.09090909090909091)
H ({u'B': 1, u'E': 1}, 0.09090909090909091)
I ({u'B': 1, u'E': 1}, 0.09090909090909091)
J ({u'E': 1}, 0.09090909090909091)
K ({u'E': 1}, 0.09090909090909091)
*total_nodes 11
A [{}, 0.09090909090909091]
B [{"C": 1}, 0.09090909090909091]
C [{"B": 1}, 0.09090909090909091]
D [{"A": 1, "B": 1}, 0.09090909090909091]
E [{"B": 1, "D": 1, "F": 1}, 0.09090909090909091]
F [{"B": 1, "E": 1}, 0.09090909090909091]
G [{"B": 1, "E": 1}, 0.09090909090909091]
H [{"B": 1, "E": 1}, 0.09090909090909091]
I [{"B": 1, "E": 1}, 0.09090909090909091]
J [{"E": 1}, 0.09090909090909091]
K [{"E": 1}, 0.09090909090909091]
dm:  0.0909090909091
dm 0.0909090909
E [{"B": 1, "D": 1, "F": 1}, 0.36363636363636365]
E ({u'B': 1, u'D': 1, u'F': 1}, 

In [1]:
#### Create HDFS directories
#!hdfs dfs -mkdir /user/hadoop/HW9Output
#!hdfs dfs -mkdir /user/hadoop/HW9Input
#!hdfs dfs -mkdir /user/hadoop/HW9Results
#!hdfs dfs -mkdir /user/hadoop/HW9Data
#!hdfs dfs -copyFromLocal PageRank-test.txt /user/hadoop/HW9Data

16/07/14 14:06:10 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Test PageRankInit with HDFS

In [26]:
import PageRank
import PageRankInit
reload(PageRank)
reload(PageRankInit)
import json

def get_total_nodes(ofile):
    ls_output = !hdfs dfs -ls $ofile
    num_output_files = len(ls_output)-3
    for i in range(num_output_files):
        snum = str(i)
        zeros = '0'*(5-len(snum))
        rfile = ofile+'/part-'+zeros+snum
        grep_output = !hdfs dfs -cat $rfile | grep '*total_nodes'
        if len(grep_output) == 2:
            _, total_nodes = grep_output[1].strip().split('\t')
            break
    return total_nodes

# Set the damping factor
damping_factor = 0.85
#damping_factor = 1.0
epsilon = 0.01

# Set the name of the file that gets passed from iteration to iteration
work_filename = 'work_table.txt'

# Initialize a list ot store the page_ranks from iteration to iteration
page_ranks = []
# Call the PageRankInit MRJob
ifile = 'hdfs:///user/hadoop/HW9Data/PageRank-test.txt'
#ifile = 'hdfs:///user/hadoop/HW7data/all-pages-indexed-out.txt'
ofile ='HW9Results/PageRankInit_test'
!hdfs dfs -rm -r /user/hadoop/$ofile
mr_job = PageRankInit.MRPageRankInit(args=[ifile, '-r', 'hadoop', '--output-dir', ofile])
with mr_job.make_runner() as runner, open(work_filename, 'w') as f: 
    runner.run()
    # stream_output: get access of the output 
    for line in runner.stream_output():
        key, value =  mr_job.parse_output_line(line)
        # Store the total_nodes
        if key == '*total_nodes':
            total_nodes = float(value)
        else:
            page_ranks.append(value[1])
        f.write(key+'\t'+json.dumps(value)+'\n')

total_nodes = get_total_nodes('/user/hadoop/'+ofile)
print 'Total nodes: ', total_nodes

16/07/14 01:18:49 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `/user/hadoop/HW9Results/PageRankInit_test': No such file or directory


ERROR:mrjob.fs.hadoop:STDERR: 16/07/14 01:19:29 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable



Total nodes:  11


In [9]:
!hdfs dfs -ls /user/hadoop/$ofile
#!hdfs dfs -cat /user/hadoop/$ofile/part-00000

16/07/13 23:39:20 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   3 hadoop supergroup          0 2016-07-13 23:38 /user/hadoop/HW9Output/PageRankInit/_SUCCESS
-rw-r--r--   3 hadoop supergroup        464 2016-07-13 23:38 /user/hadoop/HW9Output/PageRankInit/part-00000


### Create a nohup job to start the Hadoop job in the background

In [24]:
%%writefile start_job_PageRankInit.sh
#!/bin/bash
hdfs dfs -rm -r /user/hadoop/HW9Output/PageRankInit
nohup python PageRankInit.py -r hadoop hdfs:///user/hadoop/HW7data/all-pages-indexed-out.txt --output-dir=HW9Output/PageRankInit --no-output > nohup/PageRankInit.out &


Overwriting start_job_PageRankInit.sh


In [22]:
!chmod +x start_job_PageRankInit.sh

In [25]:
!./start_job_PageRankInit.sh

16/07/14 01:08:18 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `/user/hadoop/HW9Output/PageRankInit': No such file or directory


In [8]:
#!hdfs dfs -ls /user/hadoop/HW9Output/PageRankInit
#!hdfs dfs -cp /user/hadoop/HW9Output/PageRankInit /user/hadoop/HW9Results/PageRankInit_wikipedia
!hdfs dfs -cat /user/hadoop/HW9Results/PageRankInit_wikipedia/part-00000 | head
!hdfs dfs -cat /user/hadoop/HW9Results/PageRankInit_test/part-00000 | head
g = !hdfs dfs -cat /user/hadoop/HW9Results/PageRankInit_wikipedia/part-00000 | grep '*total_nodes'
print g

16/07/14 09:29:20 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
"*total_nodes"	15192277
"1"	[{}, 6.582291778908455e-08]
"10"	[{"12": 3, "14478968": 1, "14965909": 1, "5172381": 2, "3403886": 1, "8031052": 1, "6857278": 2, "9416257": 1, "14481011": 1, "12336458": 1, "994890": 1}, 6.582291778908455e-08]
"100"	[{}, 6.582291778908455e-08]
"1000"	[{}, 6.582291778908455e-08]
"10000"	[{}, 6.582291778908455e-08]
"100000"	[{"100001": 1}, 6.582291778908455e-08]
"1000000"	[{"1000001": 1}, 6.582291778908455e-08]
"10000000"	[{}, 6.582291778908455e-08]
"10000001"	[{}, 6.582291778908455e-08]
cat: Unable to write to output stream.
16/07/14 09:29:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
"*total_nodes"	11
"A"	[{}, 0.09090909090909091]
"B"	[{"C": 1}, 0.09090909090909091]
"C"	[{"B": 1}, 0.09090909090909091]
"D"	[{"A": 1, "B": 1}, 0.

### Test PageRank in HDFS

In [5]:
def get_total_nodes(ofile):
    ls_output = !hdfs dfs -ls $ofile
    num_output_files = len(ls_output)-3
    for i in range(num_output_files):
        snum = str(i)
        zeros = '0'*(5-len(snum))
        rfile = ofile+'/part-'+zeros+snum
        grep_output = !hdfs dfs -cat $rfile | grep '*total_nodes'
        if len(grep_output) == 2:
            _, total_nodes = grep_output[1].strip().split('\t')
            break
    return total_nodes

### Get the Total Nodes from the wikipedia data

In [6]:
total_nodes = get_total_nodes('/user/hadoop/HW9Results/PageRankInit_wikipedia')

In [7]:
print total_nodes

15192277


In [29]:
import PageRankStep1
import PageRankStep2
import PageRankInit
reload(PageRankStep1)
reload(PageRankStep2)
reload(PageRankInit)

iterations = 1
total_nodes = 11.0
#total_nodes = 15192277.0

# Set the damping factor
damping_factor = 0.85
num_converter = 10000000000

!hdfs dfs -rm -r /user/hadoop/HW9Output/PageRank1
!hdfs dfs -rm -r /user/hadoop/HW9Input/PageRank1
!hdfs dfs -cp /user/hadoop/HW9Results/PageRankInit_test /user/hadoop/HW9Input/PageRank1

i = 0
while(i < iterations):
    print 'Iteration ', str(i)
    # Run the PageRankStep1 MRJob
    mr_job1 = PageRankStep1.MRPageRankStep1(args=['hdfs:///user/hadoop/HW9Input/PageRank1', '-r', 'hadoop', '--num_converter', str(num_converter), '--output-dir', 'HW9Output/PageRank1'])
    with mr_job1.make_runner() as runner: 
        runner.run()
        for line in runner.stream_output():
            key, value =  mr_job.parse_output_line(line)
            print key, value

    dangling_mass = runner.counters()[0]['PageRank_Counters']['Dangling_Mass']/float(num_converter)
    print 'Dangling Mass: ', dangling_mass

    # Run the PageRankStep2 MRJob
    mr_job2 = PageRankStep2.MRPageRankStep2(args=['hdfs:///user/hadoop/HW9Output/PageRank1', '-r', 'hadoop', '--damping_factor', damping_factor, '--total_nodes', total_nodes, '--dangling_mass', dangling_mass, '--output-dir', 'HW9Output/PageRank2'])
    with mr_job2.make_runner() as runner: 
        runner.run()

    i += 1
    if (i < iterations):
        !hdfs dfs -rm -r /user/hadoop/HW9Input/PageRank1
        !hdfs dfs -mv /user/hadoop/HW9Output/PageRank2 /user/hadoop/HW9Input/PageRank1
        !hdfs dfs -rm -r /user/hadoop/HW9Output/PageRank1

16/07/14 17:13:35 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/07/14 17:13:36 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/HW9Output/PageRank1
16/07/14 17:13:37 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/07/14 17:13:38 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/HW9Input/PageRank1
16/07/14 17:13:40 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/07/14 17:13:40 WARN hdfs.DFSClient: DFSInputStream has been closed already
16/07/14 17:13:41 WARN hdfs.DFSClient: DFSInputStream has been closed already
Iteration  0


IOError: Can't handle path: hdfs:///user/hadoop/HW9Input/PageRank1

In [5]:
%%writefile PageRank_driver.py
from PageRankStep1 import MRPageRankStep1
from PageRankStep2 import MRPageRankStep2
import subprocess

iterations = 1
total_nodes = 11.0
#total_nodes = 15192277.0

# Set the damping factor
damping_factor = 0.85
num_converter = 10000000000

# Run the PageRank MRJob
try:
    subprocess.check_call(['hdfs', 'dfs', '-rm', '-r', '/user/hadoop/HW9Output/PageRank1'])
except:
    pass
try:
    subprocess.check_call(['hdfs', 'dfs', '-rm', '-r', '/user/hadoop/HW9Input/PageRank1'])
except:
    pass
subprocess.check_call(['hdfs', 'dfs', '-cp', '/user/hadoop/HW9Results/PageRankInit_test', '/user/hadoop/HW9Input/PageRank1'])
#subprocess.check_call(['hdfs', 'dfs', '-cp', '/user/hadoop/HW9Results/PageRankInit_wikipedia', '/user/hadoop/HW9Input/PageRank1'])

i = 0
while(i < iterations):
    print 'Iteration ', str(i)
    # Run the PageRankStep1 MRJob
    mr_job1 = MRPageRankStep1(args=['hdfs:///user/hadoop/HW9Input/PageRank1', '-r', 'hadoop', '--num_converter', str(num_converter), '--output-dir', 'HW9Output/PageRank1'])
    with mr_job1.make_runner() as runner: 
        runner.run()
    dangling_mass = runner.counters()[0]['PageRank_Counters']['Dangling_Mass']/float(num_converter)
    print 'Dangling Mass: ', dangling_mass

    # Run the PageRankStep2 MRJob
    mr_job2 = MRPageRankStep2(args=['hdfs:///user/hadoop/HW9Output/PageRank1', '-r', 'hadoop', '--damping_factor', damping_factor, '--total_nodes', total_nodes, '--dangling_mass', dangling_mass, '--output-dir', 'HW9Output/PageRank2'])
    with mr_job2.make_runner() as runner: 
        runner.run()

    i += 1
    if (i < iterations):
        subprocess.check_call(['hdfs', 'dfs', '-rm', '-r', '/user/hadoop/HW9Input/PageRank1'])
        subprocess.check_call(['hdfs', 'dfs', '-mv', '/user/hadoop/HW9Output/PageRank2', '/user/hadoop/HW9Input/PageRank1'])
        subprocess.check_call(['hdfs', 'dfs', '-rm', '-r', '/user/hadoop/HW9Output/PageRank1'])

Overwriting PageRank_driver.py


In [3]:
%%writefile start_job_PageRank.sh
#!/bin/bash
nohup python PageRank_driver.py > nohup/PageRank_driver.out &

Writing start_job_PageRank.sh


In [4]:
!chmod +x start_job_PageRank.sh

In [6]:
!./start_job_PageRank.sh

nohup: redirecting stderr to stdout


In [11]:
!cat nohup/PageRank_driver.out

16/07/14 17:24:31 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/07/14 17:24:32 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/HW9Output/PageRank1
16/07/14 17:24:33 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/07/14 17:24:34 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/HW9Input/PageRank1
16/07/14 17:24:35 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/07/14 17:24:36 WARN hdfs.DFSClient: DFSInputStream has been closed already
16/07/14 17:24:36 WARN hdfs.DFSClient: DFSInputStream has been closed already
No handlers could be found for logger "mrjob.

In [16]:
!hdfs dfs -cat /user/hadoop/HW9Input/PageRank1/part-00000

16/07/14 16:43:22 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
"*total_nodes"	11
"A"	[{}, 0.09090909090909091]
"B"	[{"C": 1}, 0.09090909090909091]
"C"	[{"B": 1}, 0.09090909090909091]
"D"	[{"A": 1, "B": 1}, 0.09090909090909091]
"E"	[{"B": 1, "D": 1, "F": 1}, 0.09090909090909091]
"F"	[{"B": 1, "E": 1}, 0.09090909090909091]
"G"	[{"B": 1, "E": 1}, 0.09090909090909091]
"H"	[{"B": 1, "E": 1}, 0.09090909090909091]
"I"	[{"B": 1, "E": 1}, 0.09090909090909091]
"J"	[{"E": 1}, 0.09090909090909091]
"K"	[{"E": 1}, 0.09090909090909091]


<h2 style="color:darkgreen">  HW 9.1 Analysis </h2>




<br><br>

<h2 style="color:darkblue"> HW 9.2: Exploring PageRank teleportation and network plots </h2>

* In order to overcome  problems such as disconnected components, the damping factor (a typical value for d is 0.85) can be varied. 
* Using the graph in HW1, plot the test graph (using networkx, https://networkx.github.io/) for several values of the damping parameter alpha, so that each nodes radius is proportional to its PageRank score. 
* In particular you should do this for the following damping factors: [0,0.25,0.5,0.75, 0.85, 1]. 
* Note your plots should look like the following: https://en.wikipedia.org/wiki/PageRank#/media/File:PageRanks-Example.svg

<h2 style="color:darkgreen"> HW 9.2 Implementation </h2>

In [1]:
## Code goes here

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

<h2 style="color:darkgreen">  HW 9.2 Analysis </h2>




<br><br>

<h2 style="color:darkblue"> HW 9.3: Applying PageRank to the Wikipedia hyperlinks network </h2>

* Run your PageRank implementation on the Wikipedia dataset for 5 iterations, and display the top 100 ranked nodes (with alpha = 0.85).
* Run your PageRank implementation on the Wikipedia dataset for 10 iterations, and display the top 100 ranked nodes (with teleportation factor of 0.15).
* Have the top 100 ranked pages changed? Comment on your findings. 
* Plot the pagerank values for the top 100 pages resulting from the 5 iterations run. Then plot the pagerank values for the same 100 pages that resulted from the 10 iterations run.  


<h2 style="color:darkgreen"> HW 9.3 Implementation </h2>

In [1]:
## Code goes here

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

 <h2 style="color:darkgreen">  HW 9.3 Analysis </h2>




<br><br>

<h2 style="color:darkblue"> HW 9.4: Topic-specific PageRank implementation using MRJob </h2>

Modify your PageRank implementation to produce a topic specific PageRank implementation, as described in:

http://www-cs-students.stanford.edu/~taherh/papers/topic-sensitive-pagerank.pdf

Note in this article that there is a special caveat to ensure that the transition matrix is irreducible.   
This caveat lies in footnote 3 on page 3:
```
	A minor caveat: to ensure that M is irreducible when p
	contains any 0 entries, nodes not reachable from nonzero
	nodes in p should be removed. In practice this is not problematic.
```
and must be adhered to for convergence to be guaranteed.   

Run topic specific PageRank on the following randomly generated network of 100 nodes:

> s3://ucb-mids-mls-networks/randNet.txt (also available on Dropbox)

which are organized into ten topics, as described in the file:

> s3://ucb-mids-mls-networks/randNet_topics.txt  (also available on Dropbox)

Since there are 10 topics, your result should be 11 PageRank vectors (one for the vanilla PageRank implementation in 9.1, and one for each topic with the topic specific implementation). Print out the top ten ranking nodes and their topics for each of the 11 versions, and comment on your result. Assume a teleportation factor of 0.15 in all your analyses.

One final and important comment here:  please consider the requirements for irreducibility with topic-specific PageRank. In particular, the literature ensures irreducibility by requiring that nodes not reachable from in-topic nodes be removed from the network.

This is not a small task, especially as it it must be performed separately for each of the (10) topics.

So, instead of using this method for irreducibility, please comment on why the literature's method is difficult to implement, and what what extra computation it will require.   

Then for your code, please use the alternative, non-uniform damping vector:

```
vji = beta*(1/|Tj|); if node i lies in topic Tj

vji = (1-beta)*(1/(N - |Tj|)); if node i lies outside of topic Tj
```
for beta in (0,1) close to 1. 

With this approach, you will not have to delete any nodes. If beta > 0.5, PageRank is topic-sensitive, and if beta < 0.5, the PageRank is anti-topic-sensitive. For any value of beta irreducibility should hold, so please try beta=0.99, and perhaps some other values locally, on the smaller networks.

<h2 style="color:darkgreen"> HW 9.4 Implementation </h2>

In [1]:
## Code goes here

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

<h2 style="color:darkgreen">  HW 9.4 Analysis </h2>




<br><br>

<center><div class='jumbotron'><h3 style='color:darkblue'>---------  OPTIONAL QUESTIONS SECTION --------</h3></div></center>

<h2 style="color:darkblue"> HW 9.5: (OPTIONAL) Applying topic-specific PageRank to Wikipedia</h2>

Here you will apply your topic-specific PageRank implementation to Wikipedia, defining topics (very arbitrarily) for each page by the length (number of characters) of the name of the article mod 10, so that there are 10 topics. 

* Once again, print out the top ten ranking nodes and their topics for each of the 11 versions, and comment on your result. Assume a teleportation factor of 0.15 in all your analyses. Run for 10 iterations.
* Plot the pagerank values for the top 100 pages resulting from the 5 iterations run in HW 9.3. 
* Then plot the pagerank values for the same 100 pages that result from the topic specific pagerank after 10 iterations run. 
* Comment on your findings. 

<h2 style="color:darkgreen"> HW 9.5 Implementation </h2>

In [1]:
## Code goes here

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

<h2 style="color:darkgreen">  HW 9.5 Analysis </h2>




<br><br>

<h2 style="color:darkblue"> HW 9.6:  (OPTIONAL) TextRank</h2>

* What is TextRank? Describe the main steps in the algorithm. Why does TextRank work?
* Implement TextRank in MrJob for keyword phrases (not just unigrams) extraction using co-occurrence based similarity measure with with sizes of N = 2 and 3. And evaluate your code using the following example using precision, recall, and FBeta (Beta=1):
```
"Compatibility of systems of linear constraints over the set of natural numbers
Criteria of compatibility of a system of linear Diophantine equations, strict 
inequations, and nonstrict inequations are considered. Upper bounds for
components of a minimal set of solutions and algorithms of construction of 
minimal generating sets of solutions for all types of systems are given. 
These criteria and the corresponding algorithms for constructing a minimal 
supporting set of solutions can be used in solving all the considered types of 
systems and systems of mixed types." 
```
* The extracted keywords should in the following set:
```
linear constraints, linear diophantine equations, natural numbers, non-strict inequations, strict inequations, upper bounds
```

<h2 style="color:darkgreen"> HW 9.6 Implementation </h2>

In [1]:
## Code goes here

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

<h2 style="color:darkgreen">  HW 9.6 Analysis </h2>




<br><br>

<center><div class='jumbotron'><h2 style='color:green'>-------  END OF HWK 9 --------</h2></div></center>

In [1]:
%%writefile test.py
print "Megan is awesome"

Writing test.py


In [2]:
!python test.py

Megan is awesome


In [3]:
%%writefile test_driver.py
nohup python test.py > nohup/test &

Writing test_driver.py


In [4]:
!chmod +x test_driver.py

In [6]:
!./test_driver.py

nohup: redirecting stderr to stdout


In [7]:
!cat nohup/test

Megan is awesome


In [44]:
import PageRank
import PageRankInit
reload(PageRank)
reload(PageRankInit)
import json

def get_total_nodes(ofile):
    ls_output = !hdfs dfs -ls $ofile
    num_output_files = len(ls_output)-3
    for i in range(num_output_files):
        snum = str(i)
        zeros = '0'*(5-len(snum))
        rfile = ofile+'/part-'+zeros+snum
        grep_output = !hdfs dfs -cat $rfile | grep '*total_nodes'
        if len(grep_output) == 2:
            _, total_nodes = grep_output[1].strip().split('\t')
            break
    return float(total_nodes)

# Set the damping factor
damping_factor = 0.85
iterations = 1.0

ofile ='HW9Results/PageRankInit_test'

total_nodes = get_total_nodes('/user/hadoop/'+ofile)
print total_nodes

# Run the PageRank MRJob
ifile = 'hdfs:///user/hadoop/'+ofile
ofile ='HW9Output/PageRank'
!hdfs dfs -rm -r /user/hadoop/$ofile
mr_job = PageRank.MRPageRank(args=['hdfs:///user/hadoop/HW9Results/PageRankInit_test/part-00000', '-r', 'hadoop', '--damping_factor', damping_factor, '--total_nodes', total_nodes, '--output-dir', ofile])
    
with mr_job.make_runner() as runner: 
    runner.run()
    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        print key, value
    
print 'all done'

11.0
16/07/14 01:46:29 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `/user/hadoop/HW9Output/PageRank': No such file or directory


TypeError: coercing to Unicode: need string or buffer, float found

In [ ]:
i = 0
# Set stop condition to False
Stop = False
while(False):
#while(Stop == False):
#while(i < 40):
    work_table = {}
    # Store the previous page ranks to compare of the stop conditions
    old_page_ranks = page_ranks
    page_ranks = []
    # Print the iteration number
    print('Iteration %d' % (i))
    with mr_job.make_runner() as runner: 
        runner.run()
        # stream_output: get access of the output 
        for line in runner.stream_output():
            key,value =  mr_job.parse_output_line(line)
            #print key, value
            work_table[key] = value
            page_ranks.append(value[1])
        
        # Update work_table for the next iteration
        with open(work_filename, 'w') as f:
            for key, value in work_table.iteritems():
                f.write(key+'\t'+json.dumps(value)+'\n')
        
        # Check the stop critera against the threshold epsilon
        Stop = stop_criterion_reached(old_page_ranks, page_ranks, epsilon)
        #print('Sum of page ranks: %f' % (sum(page_ranks)))
    i += 1

# Print the final results
#print page_ranks
#print('Sum of page ranks: %f' % (sum(page_ranks)))

In [ ]:
def stop_criterion_reached(old_page_ranks, page_ranks, epsilon):
    Stop = False
    total_error = 0.0
    for pr1, pr2 in zip(old_page_ranks, page_ranks):
        total_error += abs(pr1-pr2)
    print 'Total Error: ', total_error
    if total_error < epsilon:
        Stop = True
    return Stop